# Single Run

In [1]:
import numpy as np
import pandas as pd
import pickle


from surgeryschedulingunderuncertainty.task import Task
from surgeryschedulingunderuncertainty.patients_provider import PatientsFromHistoricalDataProvider
from surgeryschedulingunderuncertainty.master import Master


from surgeryschedulingunderuncertainty.implementor import StandardImplementor
from surgeryschedulingunderuncertainty.optimizer import ImplementorAdversary


from surgeryschedulingunderuncertainty.predictive_model import NGBLogNormal, NGBNormal


np.random.seed(seed = 0)


/var/folders/w_/kqymfxcs5d561j515w6_cfqh0000gn/T/ipykernel_29461/3871698272.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


## Task: pazienti storici

In [2]:
#task_file = open('task.pickle', 'rb')
task_file = open('task_100_1.pickle', 'rb')
     
# source, destination
task = pickle.load(task_file)                    
task_file.close()

or_master_schedule = pd.read_csv("../not_git/master_schedule_input_v2.csv", sep = ';')

task.master_schedule = Master(name = "my master", table=or_master_schedule)

In [3]:
or_master_schedule

,weekday,equipes,room,duration
0,1,C,or1,420
1,1,H,or2,420
2,1,D,or3,420
3,1,L,or4,420
4,1,F,or5,420
5,2,I,or1,420
6,2,C,or2,420
7,2,F,or3,420
8,2,B,or4,420
9,2,A,or5,420


In [4]:
len(task.patients)

100

In [5]:
task.patients[0].surgery_day

-1

### Implementor - Adversary

In [6]:
implementor_adversary = ImplementorAdversary(task = task, 
                                             implementor=StandardImplementor(), 
                                             adversary=None, 
                                             description = 'Test')

In [7]:
implementor_adversary.create_instance()

15
{1: 420, 2: 420, 3: 420, 4: 420, 5: 420, 6: 420, 7: 420, 8: 420, 9: 420, 10: 420, 11: 420, 12: 420, 13: 420, 14: 420, 15: 420, 16: 420, 17: 420, 18: 420, 19: 420, 20: 420, 21: 420, 22: 420, 23: 420, 24: 420, 25: 420, 26: 420, 27: 420, 28: 420, 29: 420, 30: 420, 31: 420, 32: 420, 33: 420, 34: 420, 35: 420, 36: 420, 37: 420, 38: 420, 39: 420, 40: 420, 41: 420, 42: 420, 43: 420, 44: 420, 45: 420, 46: 420, 47: 420, 48: 420, 49: 420, 50: 420, 51: 420, 52: 420, 53: 420, 54: 420, 55: 420, 56: 420, 57: 420, 58: 420, 59: 420, 60: 420, 61: 420, 62: 420, 63: 420, 64: 420, 65: 420, 66: 420, 67: 420, 68: 420, 69: 420, 70: 420, 71: 420, 72: 420, 73: 420, 74: 420, 75: 420}


In [8]:
implementor_adversary._implementor.instance_data.get(None).get('u')

{1: 12,
 2: 6,
 3: 1,
 4: 6,
 5: 12,
 6: 2,
 7: 6,
 8: 45,
 9: 12,
 10: 2,
 11: 45,
 12: 1,
 13: 12,
 14: 45,
 15: 12,
 16: 1,
 17: 6,
 18: 2,
 19: 6,
 20: 6,
 21: 45,
 22: 6,
 23: 1,
 24: 45,
 25: 45,
 26: 6,
 27: 1,
 28: 1,
 29: 6,
 30: 2,
 31: 6,
 32: 12,
 33: 1,
 34: 45,
 35: 2,
 36: 12,
 37: 12,
 38: 12,
 39: 6,
 40: 12,
 41: 12,
 42: 45,
 43: 45,
 44: 1,
 45: 6,
 46: 45,
 47: 45,
 48: 6,
 49: 6,
 50: 1,
 51: 6,
 52: 6,
 53: 1,
 54: 12,
 55: 45,
 56: 45,
 57: 12,
 58: 2,
 59: 1,
 60: 6,
 61: 1,
 62: 2,
 63: 6,
 64: 6,
 65: 1,
 66: 12,
 67: 45,
 68: 2,
 69: 12,
 70: 12,
 71: 45,
 72: 45,
 73: 45,
 74: 1,
 75: 12,
 76: 2,
 77: 45,
 78: 45,
 79: 45,
 80: 12,
 81: 2,
 82: 12,
 83: 12,
 84: 6,
 85: 12,
 86: 6,
 87: 2,
 88: 45,
 89: 2,
 90: 12,
 91: 6,
 92: 12,
 93: 1,
 94: 2,
 95: 12,
 96: 45,
 97: 2,
 98: 6,
 99: 45,
 100: 45}

In [9]:
schedule, report, solved_instance = implementor_adversary.run(15)

15
{1: 420, 2: 420, 3: 420, 4: 420, 5: 420, 6: 420, 7: 420, 8: 420, 9: 420, 10: 420, 11: 420, 12: 420, 13: 420, 14: 420, 15: 420, 16: 420, 17: 420, 18: 420, 19: 420, 20: 420, 21: 420, 22: 420, 23: 420, 24: 420, 25: 420, 26: 420, 27: 420, 28: 420, 29: 420, 30: 420, 31: 420, 32: 420, 33: 420, 34: 420, 35: 420, 36: 420, 37: 420, 38: 420, 39: 420, 40: 420, 41: 420, 42: 420, 43: 420, 44: 420, 45: 420, 46: 420, 47: 420, 48: 420, 49: 420, 50: 420, 51: 420, 52: 420, 53: 420, 54: 420, 55: 420, 56: 420, 57: 420, 58: 420, 59: 420, 60: 420, 61: 420, 62: 420, 63: 420, 64: 420, 65: 420, 66: 420, 67: 420, 68: 420, 69: 420, 70: 420, 71: 420, 72: 420, 73: 420, 74: 420, 75: 420}
implementor
adversary
checkpoint 1
checkpoint 1
checkpoint 1
checkpoint 1
checkpoint 1
checkpoint 1
checkpoint 1
checkpoint 1
checkpoint 1
checkpoint 1
checkpoint 1
checkpoint 1
checkpoint 1
checkpoint 1
checkpoint 1
checkpoint 1
checkpoint 1
checkpoint 1
checkpoint 1
checkpoint 1
checkpoint 1
checkpoint 1
checkpoint 1
checkpoin

In [10]:
report

{'total duration': 0.08405125,
 'start time': datetime.datetime(2024, 9, 8, 11, 32, 39, 439037),
 'end time': datetime.datetime(2024, 9, 8, 11, 32, 44, 482112),
 'number of iterations': 2,
 'is last robust': 'Yes',
 'iterations': [{'flag': False, 'duration': 0.024293583333333334},
  {'flag': True, 'duration': 0.05975735}]}

In [11]:
schedule

{'task description': 'My first problem',
 'creation date': '2024-09-08 11:32:44',
 'number of blocks': 75,
 'blocks': [{'patients ids': [7048, 7953],
   'patients durations (nominal)': [85.77848951435317, 85.89925592561094],
   'patients urgencies': [0, 0],
   'patients max waiting days': [7, 7],
   'patients delay in days': [0, 0],
   'patients equipe': ['C', 'C'],
   'patients surgery day': [1.0, 1.0],
   'patients z delay in days': [0, 0],
   'total duration (nominal)': 171.6777454399641,
   'block allocated duration': 420,
   'room': 'or1',
   'weekday': 1,
   'order_in_day': 1,
   'week': 0,
   'days since beginning': 0,
   'equipes': ['C'],
   'overtime probability': 0},
  {'patients ids': [5411, 8670],
   'patients durations (nominal)': [112.50292955615492, 57.183009895597856],
   'patients urgencies': [0, 0],
   'patients max waiting days': [7, 7],
   'patients delay in days': [0, 0],
   'patients equipe': ['H', 'H'],
   'patients surgery day': [1.0, 1.0],
   'patients z delay 

In [12]:
[ x.get('equipes') for x in schedule.get('blocks') if len(x.get('patients ids')) == 0]

[['A'],
 ['I'],
 ['A'],
 ['I'],
 ['A'],
 ['I'],
 ['H'],
 ['E'],
 ['I'],
 ['H'],
 ['E'],
 ['H'],
 ['I'],
 ['A'],
 ['I'],
 ['L'],
 ['B'],
 ['A'],
 ['I'],
 ['L'],
 ['H'],
 ['E'],
 ['I'],
 ['L'],
 ['H'],
 ['E']]

In [13]:
len(schedule.get('patients_not_included'))

23

In [14]:
for i in range(1, 100+1):
    print(i, sum(solved_instance.x[:, i]()), solved_instance.y[i](), solved_instance.z[i]())

1 -3.7311378230856356e-14 16.00000000000015 27.00000000000015
2 0.0 16.0 53.0
3 0.0 16.0 347.0
4 1.0 5.0 7.0
5 1.0 4.0 11.0
6 0.0 16.0 149.0
7 1.0 14.0 17.0
8 1.0 1.0 0.0
9 1.0 5.000000000000016 4.000000000000016
10 0.0 16.0 165.0
11 1.0 1.0 0.0
12 1.0 11.0 61.0
13 1.0 5.0 0.0
14 1.0 1.0 0.0
15 1.0 12.000000000000094 31.000000000000092
16 0.0 16.0 35.0
17 1.0 3.0 52.0
18 0.0 16.0 143.0
19 0.0 16.0 10.0
20 1.0 11.0 18.0
21 1.0 1.0 0.0
22 1.0 2.0 32.0
23 1.0 5.0 1.0
24 1.0 6.0 0.0
25 1.0 2.0 0.0
26 1.0 8.0 5.0
27 1.0 13.0 164.0
28 1.0 6.0 143.0
29 0.0 16.0 23.0
30 1.0 12.0 140.0
31 0.0 16.0 11.0
32 1.0 6.0 27.0
33 0.0 16.0 276.0
34 1.0 3.0 0.0
35 1.0 15.0 133.0
36 1.0 4.0 20.0
37 1.0 4.0 0.0
38 1.0 7.0 30.0
39 1.0 8.000000000000052 46.00000000000005
40 1.0 7.0 27.0
41 1.0 6.0 20.0
42 1.0 1.0 0.0
43 1.0 1.0 0.0
44 0.0 16.0 92.0
45 1.0 4.0 35.0
46 1.0 6.0 0.0
47 0.0 16.0 15.0
48 0.0 16.0 22.0
49 0.0 16.0 34.0
50 1.0 12.0 93.0
51 0.0 16.0 54.0
52 1.0 6.0 44.0
53 1.0 5.0 98.0
54 1.0 11.0 13.

In [15]:
import pandas as pd
pd.set_option('display.max_rows', None)


pd.DataFrame(schedule.get('patients_not_included'))

,id,urgency,days difference,equipe,duration nominal,uncertainty parameters,surgery_day,delay_in_days
0,4779,1,12,F,50.714658,"shape: 50.71585568249711, scale: 0.34860941449...",16.0,27.0
1,4495,2,16,F,44.370198,"shape: 44.37182363041054, scale: 0.37977806924...",16.0,53.0
2,6195,4,22,D,46.389503,"shape: 46.390545478471374, scale: 0.3110137569...",16.0,347.0
3,9130,3,40,F,48.435282,"shape: 48.43672751082199, scale: 0.37416993994...",16.0,149.0
4,5025,3,24,F,48.315897,"shape: 48.317228446102575, scale: 0.3587710218...",16.0,165.0
5,4940,4,334,F,56.499680,"shape: 56.50034531516266, scale: 0.27412864025...",16.0,35.0
6,887,3,46,F,50.405998,"shape: 50.407244587922555, scale: 0.3545170781...",16.0,143.0
7,2479,2,59,F,50.505445,"shape: 50.50669139647068, scale: 0.35478121170...",16.0,10.0
8,2936,2,46,F,50.933556,"shape: 50.934748933150466, scale: 0.3486094144...",16.0,23.0
9,1151,2,58,F,97.090520,"shape: 97.09130189849006, scale: 0.38959211178...",16.0,11.0


In [16]:
pd.DataFrame(schedule.get('blocks'))

,patients ids,patients durations (nominal),patients urgencies,patients max waiting days,patients delay in days,patients equipe,patients surgery day,patients z delay in days,total duration (nominal),block allocated duration,room,weekday,order_in_day,week,days since beginning,equipes,overtime probability
0,"[7048, 7953]","[85.77848951435317, 85.89925592561094]","[0, 0]","[7, 7]","[0, 0]","[C, C]","[1.0, 1.0]","[0, 0]",171.677745,420,or1,1,1,0,0,[C],0
1,"[5411, 8670]","[112.50292955615492, 57.183009895597856]","[0, 0]","[7, 7]","[0, 0]","[H, H]","[1.0, 1.0]","[0, 0]",169.685939,420,or2,1,2,0,0,[H],0
2,"[3817, 5944]","[45.749368608146476, 115.52705414280956]","[0, 0]","[7, 7]","[0, 0]","[D, D]","[1.0, 1.0]","[0, 0]",161.276423,420,or3,1,3,0,0,[D],0
3,[4951],[193.69234458249238],[0],[7],[0],[L],[1.0],[0],193.692345,420,or4,1,4,0,0,[L],0
4,"[8580, 2296]","[56.49962808145786, 48.14041681901115]","[0, 0]","[7, 7]","[0, 0]","[F, F]","[1.0, 1.0]","[0, 0]",104.640045,420,or5,1,5,0,0,[F],0
5,[1926],[201.19536382998893],[0],[7],[0],[I],[2.0],[0],201.195364,420,or1,2,1,0,1,[I],0
6,[7406],[131.29456909818018],[0],[7],[0],[C],[2.0],[0],131.294569,420,or2,2,2,0,1,[C],0
7,"[3393, 7620]","[50.71465580762012, 49.12172113311154]","[0, 0]","[7, 7]","[0, 0]","[F, F]","[2.0, 2.0]","[0, 0]",99.836377,420,or3,2,3,0,1,[F],0
8,"[8940, 4416]","[78.6418562791944, 77.6891517837022]","[1, 0]","[30, 7]","[0, 0]","[B, B]","[2.0, 2.0]","[0, 0]",156.331008,420,or4,2,4,0,1,[B],0
9,"[1623, 337]","[89.35691523719468, 102.79837092086541]","[2, 2]","[60, 60]","[0, 0]","[A, A]","[2.0, 2.0]","[0, 0]",192.155286,420,or5,2,5,0,1,[A],0


In [20]:
task.patients[68].__str__()

'Patient id: 8014 \n equipe: I \n urgency: 1 \n nominal duration: 348 \n days waiting: 23'

In [25]:
print(task.patients[68].uncertainty_profile._nominal_value)
print(task.patients[68].uncertainty_profile._param_s)
print(task.patients[68].uncertainty_profile._param_scale)

348.0017950655124
0.4578459042749789
348.0020962455946


In [31]:
[ (implementor_adversary._implementor.instance_data.get(None).get('eps').get(x), x) for x in implementor_adversary._implementor.instance_data.get(None).get('eps') if x[0] == 69]

[(0.0, (69, 1)),
 (0.0, (69, 2)),
 (0.0, (69, 3)),
 (0.0, (69, 4)),
 (0.0, (69, 5)),
 (0.0, (69, 6)),
 (0.0, (69, 7)),
 (0.0, (69, 8)),
 (0.0, (69, 9)),
 (0.0, (69, 10)),
 (0.0, (69, 11)),
 (60.20995165525116, (69, 12)),
 (13.90626532569209, (69, 13)),
 (106.84695539365737, (69, 14)),
 (0.0, (69, 15)),
 (0.0, (69, 16)),
 (0.0, (69, 17)),
 (0.0, (69, 18)),
 (0.0, (69, 19)),
 (0.0, (69, 20)),
 (0.0, (69, 21)),
 (0.0, (69, 22)),
 (0.0, (69, 23)),
 (0.0, (69, 24)),
 (0.0, (69, 25)),
 (0.0, (69, 26)),
 (0.0, (69, 27)),
 (0.0, (69, 28)),
 (0.0, (69, 29)),
 (0.0, (69, 30)),
 (0.0, (69, 31)),
 (0.0, (69, 32)),
 (0.0, (69, 33)),
 (0.0, (69, 34)),
 (0.0, (69, 35)),
 (0.0, (69, 36)),
 (0.0, (69, 37)),
 (0.0, (69, 38)),
 (0.0, (69, 39)),
 (0.0, (69, 40)),
 (0.0, (69, 41)),
 (0.0, (69, 42)),
 (0.0, (69, 43)),
 (0.0, (69, 44)),
 (0.0, (69, 45)),
 (0.0, (69, 46)),
 (0.0, (69, 47)),
 (0.0, (69, 48)),
 (0.0, (69, 49)),
 (0.0, (69, 50)),
 (0.0, (69, 51)),
 (0.0, (69, 52)),
 (0.0, (69, 53)),
 (0.0, (69, 

### Chance constraint

In [ ]:
from surgeryschedulingunderuncertainty.optimizer import VanillaImplementor
from surgeryschedulingunderuncertainty.implementor import ChanceConstraintsImplementor


In [ ]:
chance_constraints = VanillaImplementor(task = task, 
                                              implementor=ChanceConstraintsImplementor(task = task), 
                                              description = 'Test')

In [ ]:
chance_constraints.create_instance()

In [ ]:
#solution_chance_constraints = chance_constraints.run()
schedule, report = chance_constraints.run()

In [ ]:
len(schedule.get('patients_not_included'))

In [ ]:
schedule

In [ ]:
report

In [ ]:
[ x.get('equipes') for x in schedule.get('blocks') if len(x.get('patients ids')) == 0]

In [ ]:
len(schedule.get('patients_not_included'))

In [ ]:
pd.DataFrame(schedule.get('patients_not_included'))

In [ ]:
pd.DataFrame(schedule.get('blocks'))

### Budget Set

In [ ]:
from surgeryschedulingunderuncertainty.optimizer import  BudgetSet
from surgeryschedulingunderuncertainty.implementor import  BudgetSetImplementor


In [ ]:
task.gamma_variation = 0


budget_set = BudgetSet(task = task, 
                              implementor=BudgetSetImplementor(task = task), 
                              description = 'Test')






In [ ]:
schedule, report = budget_set.run()

In [ ]:
report

In [ ]:
schedule

In [ ]:
[ x.get('equipes') for x in schedule.get('blocks') if len(x.get('patients ids')) == 0]

In [ ]:
len(schedule.get('patients_not_included'))

In [ ]:
pd.DataFrame(schedule.get('patients_not_included'))

In [ ]:
pd.DataFrame(schedule.get('blocks'))